# Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


Training a sample Perceptron to try the blockchain integration

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# torch.set_default_device('cpu')


n_features = 10  # Example number of input features
num_classes = 1  # Example number of classes
# model = Perceptron(n_features, num_classes)  # Instantiate the model (on the default device
model = Perceptron(n_features)  # Instantiate the model (on the default device

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    
loss_function = nn.BCELoss()  # Binary Cross-Entropy Loss
# Stochastic Gradient Descent
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
    
# Example (dummy) training data
dummy_inputs = torch.randn(100, n_features)  # 100 samples, n_features each
print(dummy_inputs.device)
# Binary target values (0 or 1)
dummy_targets = torch.randint(0, 2, (100, 1)).float()

# Training loop
for epoch in range(5):  # Number of epochs
    optimizer.zero_grad()  # Clearing the gradients
    outputs = model(dummy_inputs)  # Forward pass
    loss = loss_function(outputs, dummy_targets)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Model's state_dict:
fc.weight 	 torch.Size([1, 10])
fc.bias 	 torch.Size([1])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]
cuda:0
Epoch 0, Loss: 0.821436882019043
Epoch 1, Loss: 0.8204890489578247
Epoch 2, Loss: 0.8195452094078064
Epoch 3, Loss: 0.8186054825782776
Epoch 4, Loss: 0.8176696300506592


Loading CIFAR DATASET

In [4]:
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

Files already downloaded and verified
Files already downloaded and verified


Create model and train

In [5]:
from client.model.perceptron import MultiLayerPerceptron
from client.train import train
from client.utils import weights_init, update_lr


input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
modelpath = 'client/models/'
train_flag = True

model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model.to(device)
print(model)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}

# Training
model.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=reg)

train(model, dataloaders, modelpath, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.2999
Epoch [1/1], Step [3/245], Loss: 2.2931
Epoch [1/1], Step [4/245], Loss: 2.2768
Epoch [1/1], Step [5/245], Loss: 2.2638
Epoch [1/1], Step [6/245], Loss: 2.2501
Epoch [1/1], Step [7/245], Loss: 2.2196
Epoch [1/1], Step [8/245], Loss: 2.2181
Epoch [1/1], Step [9/245], Loss: 2.2221
Epoch [1/1], Step [10/245], Loss: 2.1657
Epoch [1/1], Step [11/245], Loss: 2.1538
Epoch [1/1], Step [12/245], Loss: 2.1077
Epoch [1/1], Step [13/245], Loss: 2.0933
Epoch [1/1], Step [14/245], Loss: 2.1610
Epoch [1/1], Step [15/245], Loss: 2.0913
Epoch [1/1], Step [16/245], Loss: 2.1184
Epoch [1/1], Step [17/245], Loss: 1.9994
Epoch [1/1], Step [18/245], Loss: 1.9845
Epoch [1/1], Step [19/245], Loss: 2.0183
Epoch [1/1], Step [20/245], Loss: 2.0641

Test the trained model

In [6]:
from os.path import join as pjoin
modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
best_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
best_model = torch.load(pjoin(modelpath, 'model.ckpt'))
model.load_state_dict(best_model)
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

Accuracy of the network on the 1000 test images: 40.9 %


Send the saved model to Fabric-SDK via Gateway Client (REST call)

In [ ]:
from os.path import join as pjoin
from client.services.gateway_client import submit_local_model, get_all_models, get_model

modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
best_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
best_model = torch.load(pjoin(modelpath, 'model.ckpt'))
model.load_state_dict(best_model)

submit_local_model(model.state_dict())

Retrieve future global model and convert it again to pytorch model

In [10]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json

model_params = get_model('modelID')
# print(client_response)


model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model.to(device)
print(model)

load_model_from_json(model, model_params)
print(model)

for parameter in model.parameters():
    print(parameter)


MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
Parameter containing:
tensor([[ 0.0165,  0.0194,  0.0171,  ..., -0.0069, -0.0136, -0.0147],
        [ 0.0060,  0.0150,  0.0190,  ..., -0.0616, -0.0620, -0.0606],
        [-0.0043, -0.0144, -0.0166,  ..., -0.0392, -0.0331, -0.0260],
        ...,
        [-0.0403, -0.0381, -0.0357,  ...,  0.0240,  0.0226,  0.0221],
        [-0.0104, -0.0060, -0.0007,  ..., -0.0304, -0.0349, -0.0367],
        [-0.0251, -0.0113, -0.0075,  ..., -0.0311, -0.0307, -0.0287]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0196,  0.0870,  0.0041,  0.0843,  0.0820,  0.0826,  0.0252, -0.0290,
         0.0780, -

Create two models and aggregate them

In [17]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json, weights_zero_init
from client.aggregators import federated_aggregate

model_params = get_model('modelID')
# print(client_response)


model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
# print(model1)

load_model_from_json(model1, model_params)

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
# print(model2)
load_model_from_json(model2, model_params)


model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
weights_zero_init(model3)
# print(model3)

model_avg = federated_aggregate([model1, model3])

# print(model_avg)



for parameter in model1.parameters():
    print(parameter)
    
for parameter in model_avg.parameters():
    print(parameter)

 

MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
Parameter containing:
tensor([[ 0.0165,  0.0194,  0.0171,  ..., -0.0069, -0.0136, -0.0147],
        [ 0.0060,  0.0150,  0.0190,  ..., -0.0616, -0.0620, -0.0606],
        [-0.0043, -0.0144, -0.0166,  ..., -0.0392, -0.0331, -0.0260],
        

In [14]:
from client.utils import weights_zero_init

model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
weights_zero_init(model3)
print(model3)

for parameter in model3.parameters():
    print(parameter)

MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

Sample neural network code that checks if the tensors and model are running on GPU

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

n_features = 10  # Example number of input features
# Instantiate the model (on the default device
model = Perceptron(n_features).to('cuda')
loss_function = nn.BCELoss()  # Binary Cross-Entropy Loss
# Stochastic Gradient Descent
optimizer = optim.SGD(model.parameters(), lr=0.01)
# Example (dummy) training data
dummy_inputs = torch.randn(100, n_features)  # 100 samples, n_features each
print(dummy_inputs.device)
# Binary target values (0 or 1)
dummy_targets = torch.randint(0, 2, (100, 1)).float()

# Training loop
for epoch in range(5):  # Number of epochs
    optimizer.zero_grad()  # Clearing the gradients
    start_event.record()
    outputs = model(dummy_inputs)  # Forward pass
    end_event.record()
    torch.cuda.synchronize()  # Wait for the events to be recorded!
    elapsed_time_ms = start_event.elapsed_time(end_event)
    print(f"Elapsed time (in milliseconds): {elapsed_time_ms}")
    loss = loss_function(outputs, dummy_targets)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    if epoch % 1 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')